In [1]:
class Song:
    def __init__(self, title, artist, duration):
        self.title = title
        self.artist = artist
        self.duration = duration  # in seconds
    
    def __str__(self):
        minutes = self.duration // 60
        seconds = self.duration % 60
        return f"{self.title} - {self.artist} ({minutes}:{seconds:02d})"


class PlaylistNode:
    def __init__(self, song):
        self.song = song
        self.next = None
        self.prev = None  # For doubly linked implementation


class MusicPlaylist:
    def __init__(self, name):
        self.name = name
        self.head = None
        self.tail = None
        self.current = None  # Currently playing song
        self.size = 0
        self.total_duration = 0
    
    def add_song(self, title, artist, duration):
        """Add song to end of playlist"""
        new_song = Song(title, artist, duration)
        new_node = PlaylistNode(new_song)
        
        if self.head is None:
            self.head = new_node
            self.tail = new_node
        else:
            new_node.prev = self.tail
            self.tail.next = new_node
            self.tail = new_node
        
        self.size += 1
        self.total_duration += duration
        
        # If no current song, set to this one
        if self.current is None:
            self.current = new_node
        
        print(f"Added: {new_song}")
    
    def add_song_at_position(self, title, artist, duration, position):
        """Add song at specific position (0-based)"""
        if position < 0 or position > self.size:
            raise IndexError("Invalid position")
        
        new_song = Song(title, artist, duration)
        new_node = PlaylistNode(new_song)
        
        if position == 0:  # Insert at beginning
            new_node.next = self.head
            if self.head:
                self.head.prev = new_node
            self.head = new_node
            if self.tail is None:
                self.tail = new_node
                
        elif position == self.size:  # Insert at end
            self.add_song(title, artist, duration)
            return  # Already handled by add_song
            
        else:  # Insert in middle
            current = self.head
            for _ in range(position - 1):
                current = current.next
            
            new_node.next = current.next
            new_node.prev = current
            current.next.prev = new_node
            current.next = new_node
        
        self.size += 1
        self.total_duration += duration
        print(f"Added at position {position}: {new_song}")
    
    def remove_song(self, title):
        """Remove song by title"""
        if self.head is None:
            print("Playlist is empty")
            return False
        
        current = self.head
        
        # Check if head contains the song
        if current.song.title.lower() == title.lower():
            self.total_duration -= current.song.duration
            
            if self.current == current:
                self.current = current.next
            
            self.head = current.next
            if self.head:
                self.head.prev = None
            else:  # Playlist became empty
                self.tail = None
                self.current = None
            
            self.size -= 1
            print(f"Removed: {current.song}")
            return True
        
        # Search for song
        while current.next and current.next.song.title.lower() != title.lower():
            current = current.next
        
        if current.next is None:
            print(f"Song '{title}' not found")
            return False
        
        # Remove the node
        removed = current.next
        self.total_duration -= removed.song.duration
        
        if self.current == removed:
            self.current = removed.next or removed.prev
        
        current.next = removed.next
        
        if removed.next:
            removed.next.prev = current
        else:  # Removed the tail
            self.tail = current
        
        self.size -= 1
        print(f"Removed: {removed.song}")
        return True
    
    def play_next(self):
        """Play next song in playlist"""
        if self.current is None:
            print("No song is currently playing")
            return None
        
        if self.current.next is None:
            print("Reached end of playlist")
            return None
        
        self.current = self.current.next
        print(f"Now playing: {self.current.song}")
        return self.current.song
    
    def play_previous(self):
        """Play previous song in playlist"""
        if self.current is None:
            print("No song is currently playing")
            return None
        
        if self.current.prev is None:
            print("Reached beginning of playlist")
            return None
        
        self.current = self.current.prev
        print(f"Now playing: {self.current.song}")
        return self.current.song
    
    def shuffle(self):
        """Shuffle the playlist (Fisher-Yates algorithm)"""
        if self.size < 2:
            return
        
        # Convert to list for shuffling
        nodes = []
        current = self.head
        while current:
            nodes.append(current)
            current = current.next
        
        # Fisher-Yates shuffle
        import random
        for i in range(len(nodes) - 1, 0, -1):
            j = random.randint(0, i)
            nodes[i], nodes[j] = nodes[j], nodes[i]
        
        # Rebuild links
        self.head = nodes[0]
        self.head.prev = None
        self.tail = nodes[-1]
        self.tail.next = None
        
        for i in range(len(nodes) - 1):
            nodes[i].next = nodes[i + 1]
            nodes[i + 1].prev = nodes[i]
        
        print("Playlist shuffled!")
    
    def create_play_queue(self, start_from_current=True):
        """Create a queue of songs starting from current or beginning"""
        queue = []
        
        if start_from_current and self.current:
            current = self.current
        else:
            current = self.head
        
        while current:
            queue.append(current.song)
            current = current.next
        
        return queue
    
    def get_songs_by_artist(self, artist):
        """Get all songs by a specific artist"""
        songs = []
        current = self.head
        
        while current:
            if current.song.artist.lower() == artist.lower():
                songs.append(current.song)
            current = current.next
        
        return songs
    
    def get_playlist_info(self):
        """Get playlist statistics"""
        hours = self.total_duration // 3600
        minutes = (self.total_duration % 3600) // 60
        seconds = self.total_duration % 60
        
        return {
            'name': self.name,
            'total_songs': self.size,
            'total_duration': f"{hours}:{minutes:02d}:{seconds:02d}",
            'current_song': str(self.current.song) if self.current else "None"
        }
    
    def display_playlist(self):
        """Display all songs in playlist"""
        if self.head is None:
            print("Playlist is empty")
            return
        
        print(f"\n{'='*60}")
        print(f"PLAYLIST: {self.name}".center(60))
        print(f"{'='*60}")
        
        current = self.head
        index = 1
        
        while current:
            prefix = "▶ " if self.current == current else "  "
            print(f"{prefix}{index:3d}. {current.song}")
            current = current.next
            index += 1
        
        info = self.get_playlist_info()
        print(f"\nTotal: {info['total_songs']} songs | Duration: {info['total_duration']}")
        print(f"Now playing: {info['current_song']}")


# Example usage
def music_player_demo():
    # Create a playlist
    my_playlist = MusicPlaylist("My Favorites")
    
    # Add songs
    my_playlist.add_song("Bohemian Rhapsody", "Queen", 354)
    my_playlist.add_song("Stairway to Heaven", "Led Zeppelin", 482)
    my_playlist.add_song("Hotel California", "Eagles", 391)
    my_playlist.add_song("Imagine", "John Lennon", 183)
    my_playlist.add_song("Smells Like Teen Spirit", "Nirvana", 301)
    
    # Display playlist
    my_playlist.display_playlist()
    
    # Add song at specific position
    my_playlist.add_song_at_position("Like a Rolling Stone", "Bob Dylan", 369, 2)
    
    # Play through songs
    print("\n=== Playing songs ===")
    my_playlist.play_next()
    my_playlist.play_next()
    my_playlist.play_previous()
    
    # Get songs by artist
    print("\n=== Songs by Queen ===")
    queen_songs = my_playlist.get_songs_by_artist("Queen")
    for song in queen_songs:
        print(f"  - {song}")
    
    # Shuffle playlist
    my_playlist.shuffle()
    my_playlist.display_playlist()
    
    # Remove a song
    print("\n=== Removing song ===")
    my_playlist.remove_song("Imagine")
    
    # Create play queue
    print("\n=== Play Queue ===")
    queue = my_playlist.create_play_queue()
    for i, song in enumerate(queue, 1):
        print(f"{i:2d}. {song}")
    
    # Playlist info
    print("\n=== Playlist Info ===")
    info = my_playlist.get_playlist_info()
    for key, value in info.items():
        print(f"{key.replace('_', ' ').title()}: {value}")


if __name__ == "__main__":
    music_player_demo()

Added: Bohemian Rhapsody - Queen (5:54)
Added: Stairway to Heaven - Led Zeppelin (8:02)
Added: Hotel California - Eagles (6:31)
Added: Imagine - John Lennon (3:03)
Added: Smells Like Teen Spirit - Nirvana (5:01)

                   PLAYLIST: My Favorites                   
▶   1. Bohemian Rhapsody - Queen (5:54)
    2. Stairway to Heaven - Led Zeppelin (8:02)
    3. Hotel California - Eagles (6:31)
    4. Imagine - John Lennon (3:03)
    5. Smells Like Teen Spirit - Nirvana (5:01)

Total: 5 songs | Duration: 0:28:31
Now playing: Bohemian Rhapsody - Queen (5:54)
Added at position 2: Like a Rolling Stone - Bob Dylan (6:09)

=== Playing songs ===
Now playing: Stairway to Heaven - Led Zeppelin (8:02)
Now playing: Like a Rolling Stone - Bob Dylan (6:09)
Now playing: Stairway to Heaven - Led Zeppelin (8:02)

=== Songs by Queen ===
  - Bohemian Rhapsody - Queen (5:54)
Playlist shuffled!

                   PLAYLIST: My Favorites                   
    1. Smells Like Teen Spirit - Nirvana (5:0